In [2]:
import json
from html import unescape
import pandas as pd
import swifter

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = []
index = 0
#with open('/content/drive/MyDrive/CupIT2023/ranking_train.jsonl', 'r') as f:
with open('ranking_train.jsonl', 'r') as f:
    for line in f:
        json_data = json.loads(line.strip())
        post_text = unescape(json_data['text'])
        post_text = post_text.replace('\'', '').replace('\n', '').replace('\t', '')
        comments = json_data['comments']
        for comment in comments:
            comment_text = unescape(comment['text'])
            comment_text = comment_text.replace('\'', '').replace('\n', '').replace('\t', '')
            score = comment.get('score')
            data.append({'post_text': post_text, 'comment': comment_text, 'score': score, 'text_id': index})
        index = index + 1

df_train = pd.DataFrame(data)

In [8]:
data = []
index = 0
#with open('/content/drive/MyDrive/CupIT2023/ranking_test.jsonl', 'r') as f:
with open('ranking_test.jsonl', 'r') as f:
    for line in f:
        json_data = json.loads(line.strip())
        post_text = unescape(json_data['text'])
        post_text = post_text.replace('\'', '').replace('\n', '').replace('\t', '')
        comments = json_data['comments']
        for comment in comments:
            comment_text = unescape(comment['text'])
            comment_text = comment_text.replace('\'', '').replace('\n', '').replace('\t', '')
            score = comment.get('score')
            data.append({'post_text': post_text, 'comment': comment_text, 'text_id': index})
        index = index + 1

df_test = pd.DataFrame(data)

In [10]:
from transformers import pipeline
from transformers import AutoModelForSequenceClassification
#from optimum.bettertransformer import BetterTransformer
#model_hf = AutoModelForSequenceClassification.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
#model = BetterTransformer.transform(model_hf, keep_original_model=True)
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True, max_length=512, truncation=True)
classifier("I love this!")

C:\Users\Sergey\anaconda3\lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'disgust', 'score': 0.0016119909705594182},
  {'label': 'fear', 'score': 0.0004138523945584893},
  {'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'neutral', 'score': 0.0057645863853394985},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'surprise', 'score': 0.008528684265911579}]]

In [25]:
df_train[['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']] = None

In [26]:
df_train

,post_text,comment,score,text_id,anger,disgust,fear,joy,neutral,sadness,surprise
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0,0,None,None,None,None,None,None,None
1,How many summer Y Combinator fundees decided n...,There will invariably be those who dont see th...,1,0,None,None,None,None,None,None,None
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2,0,None,None,None,None,None,None,None
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3,0,None,None,None,None,None,None,None
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4,0,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
440530,Pay your rent with a Credit or Debit card. No ...,Most major banks offer a service called bill p...,0,88106,None,None,None,None,None,None,None
440531,Pay your rent with a Credit or Debit card. No ...,"It costs 3.25%, or $74.25 for the example of $...",1,88106,None,None,None,None,None,None,None
440532,Pay your rent with a Credit or Debit card. No ...,As many other comments have pointed out almost...,2,88106,None,None,None,None,None,None,None
440533,Pay your rent with a Credit or Debit card. No ...,My apartment building uses Yapstones RentPayme...,3,88106,None,None,None,None,None,None,None


In [11]:
def my_func(comment, classifier):
  classes = classifier(comment)
  # return scores for anger, disgust, fear, joy, neutral, sadness, surprise
  return pd.Series([classes[0][0]['score'], classes[0][1]['score'], classes[0][2]['score'], classes[0][3]['score'], classes[0][4]['score'], classes[0][5]['score'], classes[0][6]['score']]) 

In [12]:
df = df_train[0:20]

In [6]:
df['comment'][15][:2600]

'His real point is that something can be "simple and powerful." In order to sound edgy he subverts the meaning of complex into something that it doesnt mean (powerful), and bases his entire argument on this sleight of hand. (leading to statements that, out of his twisted context, are totally bewildering like "its capabilities are complex".)His argument about simplicity and complexity not being mutually exclusive is a matter of definition, and thus he undermines others by foisting his definition into their mouths and then screaming bloody murder when he deduces that changing the premises of someones argument does indeed damage its conclusions.Among other things I think his comparison of stats between Mongrel and Fogbugz is fundamentally broken. Mongrel is among other things free. Beyond the intense need to brag about his projects success I cant see any legitimate reason for making that particular comparison.(Beyond this point I rant about his usage of Hangul for his example, feel free t

In [13]:
df[['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']] = df['comment'].swifter.apply(my_func, classifier=classifier)

Pandas Apply:   0%|          | 0/20 [00:00<?, ?it/s]

C:\Users\Sergey\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [32]:
dffff_emotions = df['comment'].apply(my_func, classifier=classifier)

In [33]:
dffff_emotions

,0,1,2,3,4,5,6
0,0.688471,0.109272,0.101183,0.051947,0.034584,0.012463,0.002080
1,0.842456,0.077168,0.028417,0.021556,0.019553,0.008247,0.002604
2,0.553430,0.384262,0.036142,0.016382,0.004096,0.003367,0.002320
3,0.608035,0.152087,0.096779,0.095550,0.027339,0.013731,0.006480
4,0.447452,0.383042,0.080660,0.057461,0.015080,0.011688,0.004616
5,0.558708,0.232179,0.096711,0.095601,0.010016,0.004906,0.001879
6,0.515818,0.432804,0.021792,0.013831,0.005766,0.005011,0.004978
7,0.717742,0.199899,0.058209,0.017119,0.004202,0.002313,0.000516
8,0.601789,0.226366,0.078286,0.064289,0.014284,0.011428,0.003558
9,0.915542,0.035753,0.028377,0.010092,0.004327,0.004110,0.001798


In [9]:
df_test.head(20)

,post_text,comment,score,text_id
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0,0
1,How many summer Y Combinator fundees decided n...,There will invariably be those who dont see th...,1,0
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2,0
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3,0
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4,0
5,CBS acquires last.fm for $280m,It will be curious to see where this heads in ...,0,1
6,CBS acquires last.fm for $280m,Does this mean that theres now a big-name comp...,1,1
7,CBS acquires last.fm for $280m,Also on BBC News: http://news.bbc.co.uk/1/low...,2,1
8,CBS acquires last.fm for $280m,I dont understand what they do that is worth $...,3,1
9,CBS acquires last.fm for $280m,sold out too cheaply. given their leadership p...,4,1


In [8]:
%pip install swifter

Note: you may need to restart the kernel to use updated packages.


In [14]:
import time

In [61]:
n_start = 400000
n_end = 450000

start = time.time() ## точка отсчета времени

##код программы
df_emotions = df_train.loc[n_start:n_end, ('comment')].apply(my_func, classifier=classifier)

end = time.time() - start ## собственно время работы программы

print(end) ## вывод времени



2089.6283428668976


In [39]:
df_train.loc[n_start:n_end, ('comment')]

10000    Just a few comments about bioinformatics:- mos...
10001    Ive found Dr. Richard Hammings talk to scienti...
10002    Hi there.  If you have a good idea, then do it...
10003    "marketing + some pretty interface"Best descri...
10004    So, on a daily basis, what piece of software r...
                               ...                        
19995    Be aware of a problem here. If you go back sev...
19996    Im a big fan of the grunge trend were starting...
19997    With mobile devices growing - I think the webs...
19998    Aza Raskin is developing some really cool tech...
19999    A short while back, the folks at Adaptive Path...
Name: comment, Length: 10000, dtype: object

In [62]:
df_train.loc[n_start:n_end, ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']] = df_emotions.values

In [44]:
df_train.loc[10000,:]

post_text                Ideas for a non-web software startup?
comment      Just a few comments about bioinformatics:- mos...
score                                                        0
text_id                                                   2000
anger                                                 0.102103
disgust                                               0.122266
fear                                                 0.0159001
joy                                                  0.0758299
neutral                                               0.303799
sadness                                               0.283635
surprise                                             0.0964677
Name: 10000, dtype: object

In [45]:
df_train.loc[10000,'comment']

'Just a few comments about bioinformatics:- most of the work to be done is the same boring stuff you do at a startup: maintain the server farm, deal with flaky libraries, tune the database. The interesting algorithm work is done by PhDs... at universities.- the bullshit factor at bioinformatics companies is as high or higher than the trivial startups you mention. You are ultimately working on tools that people dont really know how to use and trying to sell them to people that probably dont need them. The only "real" customers are pharma companies or larger biotech firms. This makes the environment much like working for an enterprise software vendor which everyone knows isnt very fun.- the bioinformatics companies are either in glamorous places like New Jersey, Boston suburbs or... Silicon Valley. So, youre unlikely to be able to escape from the SV scene anyway.- at a startup these days, you at least can use slightly less painful tools like ruby and python. In bioinformatics, you are st

In [63]:
df_train.to_json(path_or_buf='df_train_emotions.json', orient='records' , lines=True)

In [ ]:
splitter = GroupShuffleSplit(test_size=.20, n_splits=1, random_state = 9)
split = splitter.split(df_train, groups=df_train['text_id'])
train_inds, test_inds = next(split)

In [ ]:
X_train = train_comment_embs[train_inds]
X_test = train_comment_embs[test_inds]
y_train = scores[train_inds]
y_test = scores[test_inds]

In [35]:
%pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [37]:
%pip install bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [38]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

model_name = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, top_k=None, max_length=512)
model_8bit = AutoModelForSequenceClassification.from_pretrained(model_name, device_map="auto", load_in_8bit=True)

text = "Hello, my llama is cute"
inputs = tokenizer(text, return_tensors="pt").to("cuda")
generated_ids = model_8bit.generate(**inputs)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(outputs)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


NameError: name 'init_empty_weights' is not defined

In [64]:
from collections import Counter

# Объединяем текстовые данные в одну строку
all_text = ' '.join(grouped_df['post_lemmatized_tokens']) + ' ' + ' '.join(grouped_df['comment_lemmatized_tokens'])

# Разделяем строку на слова
words = all_text.split()

# Считаем частоту каждого слова и сохраняем результаты в словаре
word_freq = Counter(words)

# Получаем список кортежей (слово, частота) отсортированных по убыванию частоты
sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

# Выводим первые 10 самых часто встречающихся слов
for word, freq in sorted_word_freq[:100]:
    print(f"{word}: {freq}")

,post_text,comment,score,text_id,anger,disgust,fear,joy,neutral,sadness,surprise
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0,0,0.0519469,0.109272,0.688471,0.00207975,0.101183,0.0345836,0.0124632
1,How many summer Y Combinator fundees decided n...,There will invariably be those who dont see th...,1,0,0.0771676,0.0284171,0.00260396,0.0195526,0.842456,0.00824709,0.0215558
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2,0,0.0163821,0.00336731,0.00409615,0.0361422,0.55343,0.00232044,0.384262
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3,0,0.09555,0.0273386,0.0967791,0.152087,0.608035,0.0137307,0.00647972
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4,0,0.0150801,0.0116878,0.00461623,0.383042,0.447452,0.0574614,0.0806604
...,...,...,...,...,...,...,...,...,...,...,...
440530,Pay your rent with a Credit or Debit card. No ...,Most major banks offer a service called bill p...,0,88106,0.00648713,0.00240719,0.00339717,0.00765365,0.906639,0.00874275,0.0646736
440531,Pay your rent with a Credit or Debit card. No ...,"It costs 3.25%, or $74.25 for the example of $...",1,88106,0.00915641,0.00418629,0.00137715,0.0213622,0.919053,0.00811244,0.0367525
440532,Pay your rent with a Credit or Debit card. No ...,As many other comments have pointed out almost...,2,88106,0.179334,0.122756,0.0801719,0.00384685,0.0985786,0.401896,0.113416
440533,Pay your rent with a Credit or Debit card. No ...,My apartment building uses Yapstones RentPayme...,3,88106,0.00498099,0.00201901,0.00122258,0.00192753,0.0111045,0.00610387,0.972642


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordcloud = WordCloud(width=800, height=400, background_color="white", max_words=50).generate(all_text)

# Отображаем график
plt.figure(figsize=(12, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [65]:
df = df_train[0:1000]

In [91]:
answer = df[['score','anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']]

In [95]:
ans = answer.groupby(by=['score']).mean()

DataError: No numeric types to aggregate

In [106]:
df = df.convert_dtypes(convert_string=False)

In [111]:
df.dtypes

post_text     object
comment       object
score          Int64
text_id        Int64
anger        float64
disgust      float64
fear         float64
joy          float64
neutral      float64
sadness      float64
surprise     float64
dtype: object

In [112]:
df_train = df_train.convert_dtypes(convert_string=False)

In [115]:
df_train.groupby('score')['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise'].mean()

,anger,disgust,fear,joy,neutral,sadness,surprise
score,,,,,,,
0,0.075169,0.078037,0.054015,0.074328,0.488778,0.061164,0.168509
1,0.066911,0.071148,0.047666,0.084094,0.495312,0.060381,0.174488
2,0.061767,0.067082,0.044118,0.092130,0.500216,0.057950,0.176736
3,0.059305,0.065106,0.040241,0.095801,0.503939,0.058240,0.177368
4,0.058012,0.064441,0.038666,0.097852,0.501937,0.059710,0.179382
